In [1]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import numpy as np
from sklearn import preprocessing, cluster, decomposition
from scipy.cluster.vq import kmeans,vq
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.decomposition import FastICA, PCA
from sklearn import metrics
import matplotlib.pyplot as plt
from sets import Set

riotapi.set_region("OCE")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.set_rate_limits((10, 10), (500, 600))
riotapi.print_calls(False)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: DeprecationWarning: the sets module is deprecated


In [2]:
client = MongoClient()
loladb = client.loladb
matches_collection = loladb.matches
print matches_collection.count()

28361


In [3]:
champions = riotapi.get_champions()
dict_id2idx = dict()
dict_idx2id = dict()
dict_id2key = dict()
for a_idx, a_champion in enumerate(champions):
    champ_id = a_champion.id
    champ_key = a_champion.key
    dict_id2idx[champ_id] = a_idx
    dict_idx2id[a_idx] = champ_id
    dict_id2key[champ_id] = champ_key
    
#print dict_id2key[dict_idx2id[132]]
#print dict_id2key

def GetIdxFromId(a_id):
    return dict_id2idx[a_id]

def GetIdFromIdx(a_idx):
    return dict_idx2id[a_idx]

def GetNameFromId(a_id):
    return dict_id2key[a_id]

In [4]:
items = riotapi.get_items()
items_id_to_name = {}
items_name_to_id = {}
for a_item in items:
    item_id = a_item.id
    item_name = a_item.name
    #print item_name
    items_id_to_name[item_id] = item_name
    items_name_to_id[item_name] = item_id
print items_id_to_name

{3072: u'The Bloodthirster', 2049: u'Sightstone', 2050: u"Explorer's Ward", 2051: u"Guardian's Horn", 2052: u'Poro-Snax', 2053: u'Raptor Cloak', 2054: u'Diet Poro-Snax', 2055: u'Control Ward', 1033: u'Null-Magic Mantle', 3082: u"Warden's Mail", 3083: u"Warmog's Armor", 3084: u"Overlord's Bloodmail", 3085: u"Runaan's Hurricane", 1038: u'B. F. Sword', 3599: u'The Black Spear', 3089: u"Rabadon's Deathcap", 3090: u"Wooglet's Witchcap", 3091: u"Wit's End", 3092: u"Frost Queen's Claim", 3094: u'Rapid Firecannon', 3096: u"Nomad's Medallion", 3097: u"Targon's Brace", 3098: u'Frostfang', 1051: u"Brawler's Gloves", 1052: u'Amplifying Tome', 1053: u'Vampiric Scepter', 1054: u"Doran's Shield", 3077: u'Tiamat', 3104: u"Lord Van Damm's Pillager", 1057: u'Negatron Cloak', 1058: u'Needlessly Large Rod', 3107: u'Redemption', 3108: u'Fiendish Codex', 3078: u'Trinity Force', 3110: u'Frozen Heart', 3111: u"Mercury's Treads", 3112: u"Guardian's Orb", 3113: u'Aether Wisp', 3114: u'Forbidden Idol', 1031: u'C

In [5]:
def IsCaredItem(item):
    if item.startswith('Total Biscuit'):
        return False
    if item.startswith('Warding Totem'):
        return False
    if item.startswith('Health Potion'):
        return False
    if item.startswith('Control Ward'):
        return False
    if item.startswith('Sweeping Lens'):
        return False
    if item.startswith('Farsight Alteration'):
        return False
    if item.startswith('Oracle Alteration'):
        return False
    return True

In [6]:
champions_use_num = {}
champions_kill_at_levels = [{}, {}, {}, {}, {},  {}, {}, {}, {}, {},  {}, {}, {}, {}, {},  {}, {}, {}]#18 levels
champions_level_num = {} #sum up the level that champions reached at each game 
def AddToDict(a_dict, a_key, a_value):
    if a_key in a_dict.keys():
        a_dict[a_key] += a_value
    else:
        a_dict[a_key] = a_value

def RecordChampionsUse(champions, levels):
    for a_idx, a_champ in enumerate(champions):
        AddToDict(champions_use_num, a_champ, 1)
        a_level = levels[a_idx]
        if a_champ not in champions_level_num.keys():
            champions_level_num[a_champ] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0]
        for idx in range(a_level):
            if idx == a_level - 1:
                champions_level_num[a_champ][idx] += 0.5
            else:
                champions_level_num[a_champ][idx] += 1
    
def RecordKillEvent(killer, skills):
    skill_level = 0
    #print skills
    for each_slot in range(4):
        if each_slot < 3:
            if skills[each_slot] <= 5:
                skill_level += skills[each_slot]
            else:
                skill_level += 5
        else:
            if skills[each_slot] <= 3:
                skill_level += skills[each_slot]
            else:
                skill_level += 3
    #print skill_level, killer
    if skill_level >= 1:
        AddToDict(champions_kill_at_levels[skill_level - 1], killer, 1)

In [7]:
for a_idx, a_match in enumerate(matches_collection.find()):
    #check the champs appear in this game
    participants = a_match['participants']
    champions = []
    championLevels = []
    championsItems = []
    championsSkills = []
    for a_participant in participants:
        champions.append(GetNameFromId(a_participant['championId']))
        championLevels.append(a_participant['stats']['champLevel'])
        items = Set()
        championsItems.append(items)
        championsSkills.append([0, 0, 0, 0])
    #print champions
    
    RecordChampionsUse(champions, championLevels)
    #check events
    frames = a_match['timeline']['frames']
    for a_frame_idx, a_frame in enumerate(frames):
        #print 'Frame:' + str(a_frame_idx)
        events = a_frame['events']
        for a_event in events:
            a_event_type = a_event['eventType']
            #print a_event_type
            if a_event_type == 'ITEM_PURCHASED':
                a_event_item = items_id_to_name[a_event['itemId']]
                #print a_event_item + str(a_event['itemId'])
                a_event_participantId = a_event['participantId']
                #print a_event_participantId
                if IsCaredItem(a_event_item):
                    championsItems[a_event_participantId - 1].add(a_event_item)
            if a_event_type == 'ITEM_UNDO':
                if a_event['itemBefore'] != 0:
                    a_event_item = items_id_to_name[a_event['itemBefore']]
                    a_event_participantId = a_event['participantId']
                    if IsCaredItem(a_event_item) and a_event_item in championsItems[a_event_participantId - 1]:
                        championsItems[a_event_participantId - 1].remove(a_event_item)
                if a_event['itemAfter'] != 0:
                    a_event_item = items_id_to_name[a_event['itemAfter']]
                    a_event_participantId = a_event['participantId']
                    if IsCaredItem(a_event_item):
                        championsItems[a_event_participantId - 1].add(a_event_item)                   
            if a_event_type == 'ITEM_DESTROYED' or a_event_type == 'ITEM_SOLD':
                a_event_item_id = a_event['itemId']
                if a_event_item_id != 1501:
                    a_event_item = items_id_to_name[a_event_item_id]
                    a_event_participantId = a_event['participantId']
                    if IsCaredItem(a_event_item) and a_event_item in championsItems[a_event_participantId - 1]:
                        championsItems[a_event_participantId - 1].remove(a_event_item)
            if a_event_type == 'SKILL_LEVEL_UP':
                a_event_participantId = a_event['participantId']
                a_event_skill = a_event['skillSlot']
                championsSkills[a_event_participantId - 1][a_event_skill - 1] += 1
            if a_event_type == 'CHAMPION_KILL':
                a_killer_id = a_event['killerId']
                RecordKillEvent(champions[a_killer_id - 1], championsSkills[a_killer_id - 1])
                count_in_assist = True
                if count_in_assist:
                    for a_assist_id in a_event['assistingParticipantIds']:
                        #print championsSkills[a_assist_id - 1]
                        RecordKillEvent(champions[a_assist_id - 1], championsSkills[a_assist_id - 1])
                
    #print champions                
    #print championsItems
    #print championsSkills
    #break
    if a_idx % 1000 == 0:
        print a_idx

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [8]:
#print champions_use_num
#print champions_kill_at_levels

from operator import itemgetter
sorted_champs = sorted(champions_use_num.items(), key=itemgetter(1))

for each_champ in champions_use_num:
    if each_champ in champions_level_num.keys():
        print (each_champ, champions_level_num[each_champ][13])
    
print sorted_champs

(u'MonkeyKing', 167.0)
(u'Jax', 927.5)
(u'Shaco', 356.0)
(u'Warwick', 10.5)
(u'Nidalee', 685.0)
(u'Zyra', 614.0)
(u'Kled', 905.0)
(u'Brand', 170.5)
(u'Rammus', 207.5)
(u'Illaoi', 95.5)
(u'Corki', 697.5)
(u'Braum', 287.0)
(u'Anivia', 294.0)
(u'Tryndamere', 88.5)
(u'MissFortune', 361.0)
(u'Yorick', 108.0)
(u'Xerath', 432.5)
(u'Sivir', 685.5)
(u'Riven', 913.0)
(u'Orianna', 2345.0)
(u'Gangplank', 1064.0)
(u'Malphite', 423.0)
(u'Poppy', 3827.0)
(u'Karthus', 963.0)
(u'Jayce', 3495.0)
(u'Blitzcrank', 765.0)
(u'Trundle', 634.0)
(u'Sejuani', 60.5)
(u'Graves', 3120.5)
(u'Lucian', 1149.5)
(u'Gnar', 290.0)
(u'Lux', 434.5)
(u'Shyvana', 603.5)
(u'Renekton', 681.0)
(u'Fiora', 1489.5)
(u'Jinx', 785.5)
(u'Kalista', 97.0)
(u'Fizz', 824.5)
(u'Kassadin', 512.5)
(u'Sona', 98.5)
(u'Irelia', 1011.0)
(u'Viktor', 4244.5)
(u'Kindred', 187.5)
(u'Cassiopeia', 2395.0)
(u'Maokai', 2509.5)
(u'Thresh', 2091.5)
(u'Kayle', 421.0)
(u'Hecarim', 1227.5)
(u'Khazix', 2352.0)
(u'Olaf', 787.5)
(u'Ziggs', 81.0)
(u'Syndra', 210

In [28]:
#将数据写入表格
#每一行的数据是一个英雄在1～18级时的击杀率
#一共18列代表1～18级
#每一个cell是该英雄在对应等级的归一化击杀率
import csv
output_file_name = "./ChampionKillAssistTable623Abs.csv"

with open(output_file_name, 'wb') as csvfile:
    csvfile.write(u'\ufeff'.encode('utf8'))
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    champions_kill_rate = np.zeros((len(champions_use_num), 18))
    
    for champ_idx, a_champ in enumerate(champions_use_num):
        for level in range(18):
            #该等级场均击杀率
            kill_number_at_level = 0
            if a_champ in champions_kill_at_levels[level].keys():
                kill_number_at_level = champions_kill_at_levels[level][a_champ]
            use_number_at_level = champions_level_num[a_champ][level]
            kill_rate_at_level = 0
            if use_number_at_level != 0:
                kill_rate_at_level = kill_number_at_level / float(use_number_at_level)
            champions_kill_rate[champ_idx, level] = kill_rate_at_level
            
    for champ_idx, a_champ in enumerate(champions_use_num):
        print a_champ
        a_row = [a_champ + '(' + str(champions_use_num[a_champ]) + ')']
        for level in range(18):
            #归一化等级场均击杀率
            kill_rate = champions_kill_rate[champ_idx, level]
            kill_rate_sum = np.sum(champions_kill_rate, axis = 0)[level]
            normalized_kill_rate = kill_rate / kill_rate_sum
            #a_row.append(normalized_kill_rate)
            a_row.append(kill_rate)
            #a_row.append(normalized_kill_rate)
        csvwriter.writerow(a_row)

MonkeyKing
Jax
Shaco
Warwick
Nidalee
Zyra
Kled
Brand
Rammus
Illaoi
Corki
Braum
Anivia
Tryndamere
MissFortune
Yorick
Xerath
Sivir
Riven
Orianna
Gangplank
Malphite
Poppy
Karthus
Jayce
Blitzcrank
Trundle
Sejuani
Graves
Lucian
Gnar
Lux
Shyvana
Renekton
Fiora
Jinx
Kalista
Fizz
Kassadin
Sona
Irelia
Viktor
Kindred
Cassiopeia
Maokai
Thresh
Kayle
Hecarim
Khazix
Olaf
Ziggs
Syndra
DrMundo
Karma
Annie
Akali
Leona
Yasuo
Kennen
Rengar
Ryze
Shen
Zac
Pantheon
Swain
Bard
Sion
Vayne
Nasus
Fiddlesticks
TwistedFate
Chogath
Udyr
Morgana
Ivern
Volibear
Caitlyn
Darius
Nocturne
Zilean
Azir
Rumble
Taliyah
Teemo
Urgot
Amumu
Galio
Heimerdinger
Ashe
Velkoz
Singed
Skarner
Varus
Twitch
Garen
Nunu
MasterYi
Elise
Alistar
Katarina
Ekko
Mordekaiser
KogMaw
Camille
Aatrox
Draven
TahmKench
Talon
XinZhao
AurelionSol
LeeSin
Taric
Malzahar
Lissandra
Diana
Tristana
RekSai
Vladimir
JarvanIV
Nami
Jhin
Soraka
Veigar
Janna
Nautilus
Evelynn
Gragas
Zed
Vi
Lulu
Ahri
Quinn
Leblanc
Ezreal


In [69]:
test = np.zeros((2, 3))
test[0, 0] = 1
test[1, 0] = 2
print np.sum(test, axis = 0)

[ 3.  0.  0.]


In [76]:
print champions_kill_at_levels[17]

{u'MonkeyKing': 83, u'Jax': 719, u'Shaco': 187, u'Warwick': 7, u'Nidalee': 432, u'Zyra': 79, u'Kled': 661, u'Brand': 31, u'Rammus': 83, u'Illaoi': 67, u'Corki': 458, u'Braum': 29, u'Anivia': 176, u'Tryndamere': 68, u'MissFortune': 143, u'Blitzcrank': 82, u'Yorick': 58, u'Xerath': 480, u'Sivir': 508, u'Riven': 688, u'Orianna': 1833, u'Gangplank': 1124, u'Malphite': 226, u'Poppy': 2405, u'Karthus': 943, u'Diana': 61, u'Trundle': 425, u'Sejuani': 31, u'Graves': 2162, u'Morgana': 154, u'Gnar': 185, u'Lux': 322, u'Shyvana': 366, u'Renekton': 457, u'Fiora': 983, u'Jinx': 433, u'Kalista': 37, u'Fizz': 497, u'Kassadin': 261, u'Sona': 21, u'Vladimir': 218, u'Viktor': 6914, u'Kindred': 67, u'Cassiopeia': 1802, u'Maokai': 1545, u'Thresh': 209, u'Kayle': 329, u'Hecarim': 662, u'Khazix': 2588, u'Olaf': 407, u'Ziggs': 59, u'Syndra': 1632, u'DrMundo': 41, u'Karma': 509, u'Annie': 181, u'Akali': 143, u'Leona': 14, u'Yasuo': 769, u'Kennen': 1018, u'Rengar': 691, u'Shen': 357, u'Zac': 203, u'Talon': 229